<b>1. Import libraries</b>

In [1]:
from pytrends.request import TrendReq
import pandas as pd
import plotly
import plotly.express as px
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt

%matplotlib inline 

from bokeh.io import output_notebook, show
output_notebook()

Loading BokehJS ...

<b>2. Initialize</b>

In [2]:
pytrends = TrendReq()

<b>3. General settings</b>

In [3]:
keywords = ['python']
cat = '0'
geo = 'ID'
gprop = 'youtube'

timeframes = [
    'today 5-y',
    'today 3-y',
    'today 12-m',
    'today 5-m',
    'today 3-m', 
    'today 1-m',
    'now 1-d',
    'now 7-d',
    'now 1-H',
    'now 4-H'
]

In [4]:
pytrends.build_payload(keywords, geo=geo, gprop=gprop, timeframe=timeframes[4])

<b>4. Timeseries trends data</b>

In [5]:
timeseries = pytrends.interest_over_time()
timeseries.reset_index(inplace=True)
timeseries.rename(columns={'date':'datetime', keywords[0]: 'value', 'isPartial':'partial'}, inplace=True)
timeseries.head()

,datetime,value,partial
0,2021-12-01,39,False
1,2021-12-02,76,False
2,2021-12-03,48,False
3,2021-12-04,32,False
4,2021-12-05,53,False


In [6]:
from bokeh.plotting import figure, show
from bokeh.models import DatetimeTickFormatter, NumeralTickFormatter
from bokeh.layouts import column
from bokeh.models import HoverTool

# prepare some data
x = timeseries.datetime
y = timeseries.value

hover = HoverTool(
        tooltips=[
            ("Value", "$y")
        ]
    )

# create a new plot with a specific size
p = figure(
    title=f"YouTube Trends ({keywords[0].title()}) \n",
    width=700,
    height=350,
    x_axis_label="",
    y_axis_label="",
    tools=[hover]
)



# add line renderer
circle = p.line(x, y)

# datetime custom format
p.xaxis[0].formatter = DatetimeTickFormatter(days=["%b %d, %Y"])

# column scaling
# column(p, sizing_mode='scale_width')

# show the results
show(p)

In [7]:
from bokeh.layouts import column
from bokeh.models import CustomJS, ColumnDataSource, Slider

x = timeseries.datetime
y = timeseries.value

source = ColumnDataSource(data=dict(x=x, y=y))

plot = figure(plot_width=700, plot_height=400)
plot.line('x', 'y', source=source, line_width=3, line_alpha=0.6)

slider = Slider(start=0.1, end=6, value=1, step=.1, title="power")

update_curve = CustomJS(args=dict(source=source, slider=slider), code="""
    var data = source.data;
    var f = slider.value;
    var x = data['x']
    var y = data['y']
    for (var i = 0; i < x.length; i++) {
        y[i] = Math.pow(x[i], f)
    }
    
    // necessary becasue we mutated source.data in-place
    source.change.emit();
""")

update_timeframe = CustomJS()

slider.js_on_change('value', update_curve)


show(column(slider, plot))

In [39]:
import numpy as np

from bokeh.io import show
from bokeh.layouts import row
from bokeh.models import CheckboxGroup, CustomJS
from bokeh.palettes import Viridis3
from bokeh.plotting import figure

p = figure()
props = dict(line_width=4, line_alpha=0.7)
x = np.linspace(0, 4 * np.pi, 100)
l0 = p.line(x, np.sin(x), color=Viridis3[0], legend_label="Line 0", **props)
l1 = p.line(x, 4 * np.cos(x), color=Viridis3[1], legend_label="Line 1", **props)
l2 = p.line(x, np.tan(x), color=Viridis3[2], legend_label="Line 2", **props)

checkbox = CheckboxGroup(labels=["Line 0", "Line 1", "Line 2"],
                         active=[0, 1, 2], width=100)

callback = CustomJS(args=dict(l0=l0, l1=l1, l2=l2, checkbox=checkbox), code="""
l0.visible = 0 in checkbox.active;
l1.visible = 1 in checkbox.active;
l2.visible = 2 in checkbox.active;
""")

checkbox.js_on_change('active', callback)

layout = row(checkbox, p)
show(layout)

In [ ]:
import datetime
from os.path import dirname, join

import pandas as pd
from scipy.signal import savgol_filter

from bokeh.io import curdoc
from bokeh.layouts import column, row
from bokeh.models import ColumnDataSource, DataRange1d, Select
from bokeh.palettes import Blues4
from bokeh.plotting import figure
from bokeh.io import show


timeframes = [
    'today 5-y',
    'today 3-y',
    'today 12-m',
    'today 5-m',
    'today 3-m', 
    'today 1-m',
    'now 1-d',
    'now 7-d',
    'now 1-H',
    'now 4-H'
]

activity_list = timeframes
# activity_list=df['Activity'].unique().tolist().copy()
timeseries['datetime']=pd.to_datetime(timeseries['datetime'])

activity_selected='A'

def modify_doc(doc):
    df_r = timeseries.copy()
    x = df_r.datetime
    y = df_r.value
    source = ColumnDataSource(data=df_r)
    plot = figure(title='Daily Hours', x_axis_type="datetime")
    plot.line('x', 'y', source=source, line_width=3, line_alpha=0.6)
    
    def update_plot(attr, old, new):
        activity =  select.value
        data = [df_r['value'] == activity]
        source.data = ColumnDataSource(data=data).data

    select = Select(title='Select Activity', value='today 5-y', options=activity_list)
    select.on_change('value', update_plot)

    layout=column(select, plot)
    doc.add_root(layout)
    
show(modify_doc)

In [ ]:
from bokeh.models import ColumnDataSource
from bokeh.models.widgets import Select
from bokeh.plotting import figure, output_notebook, show
import pandas as pd

output_notebook()

cats=["one","two","three"]

result={}

result['one'] = ['a','b','c']
result['two'] = ['d','e','f']
result['three'] = ['g','h','i']

origin = ColumnDataSource(result)

source = ColumnDataSource(data = dict(subcats=[]))

callback = Callback(args=dict(source=source), code="""
var f = cb_obj.get('value');
var r = origin.get('data')[f];
var data = source.get('data');
var k = data['subcats'];
for (i = 0; i < r.length; i++) {
k[i] = r[i]
}
alert(source.get('data')['subcats']);
source.trigger('change');
""")

callback.args["origin"] = origin

select1 = Select(title="Cat1", value=cats[0], options=cats, callback=callback)

select2 = Select(title="Cat2", value="", options=source.data['subcats'])

layout = column(select1, select2)

show(layout)

In [ ]:
import numpy as np

from bokeh.models import ColumnDataSource, Slider
from bokeh.plotting import figure, curdoc
from bokeh.layouts import row, column

from ipywidgets_bokeh import IPyWidget

import ipywidgets as ipw
import ipyvolume as ipv

bk_plot = figure(
    toolbar_location=None, width=200, height=200,
    x_range=(-1, 5), y_range=(-1, 1),
    x_axis_location=None, y_axis_location=None,
)
source = ColumnDataSource(data=dict(
    x=[0, 2, 4],
    angles=[0, 0, 0],
    colors=["red", "green", "blue"],
))
bk_plot.annular_wedge("x", 0, 0, 0.9, 0, dict(field="angles", units="deg"), fill_color="colors", line_color=None, source=source)

x, y, z = np.random.random((3, 1000))
ipv.quickscatter(x, y, z, size=1, marker="sphere")
ipv_plot = ipv.current.figure

bk_x = Slider(start=0, end=360, value=0, step=1, title="x")
bk_y = Slider(start=0, end=360, value=0, step=1, title="y")
bk_z = Slider(start=0, end=360, value=0, step=1, title="z")
bk_vbox = column([bk_x, bk_y, bk_z])

ip_x = ipw.FloatSlider(min=0, max=360, value=0, step=1, description="x")
ip_y = ipw.FloatSlider(min=0, max=360, value=0, step=1, description="y")
ip_z = ipw.FloatSlider(min=0, max=360, value=0, step=1, description="z")

def randomize(button):
    x, y, z = np.random.random((3, 1000))
    scatter = ipv_plot.scatters[0]
    with ipv_plot.hold_sync():
        scatter.x = x
        scatter.y = y
        scatter.z = z

ip_randomize = ipw.Button(description="Randomize")
ip_randomize.on_click(randomize)

ip_vbox = ipw.VBox([ip_x, ip_y, ip_z, ip_randomize])

def change_anglex(change):
    bk_x.value = round(np.degrees(change["new"]))
def change_angley(change):
    bk_y.value = round(np.degrees(change["new"]))
def change_anglez(change):
    bk_z.value = round(np.degrees(change["new"]))
ipv_plot.observe(change_anglex, names="anglex")
ipv_plot.observe(change_angley, names="angley")
ipv_plot.observe(change_anglez, names="anglez")

def change_bk_x(_attr, _old, new):
    ip_x.value = new
    ipv_plot.anglex = np.radians(new)
    source.patch(dict(angles=[(0, new)]))
def change_bk_y(_attr, _old, new):
    ip_y.value = new
    ipv_plot.angley = np.radians(new)
    source.patch(dict(angles=[(1, new)]))
def change_bk_z(_attr, _old, new):
    ip_z.value = new
    ipv_plot.anglez = np.radians(new)
    source.patch(dict(angles=[(2, new)]))
bk_x.on_change("value", change_bk_x)
bk_y.on_change("value", change_bk_y)
bk_z.on_change("value", change_bk_z)

def change_ip_x(change):
    bk_x.value = change["new"]
def change_ip_y(change):
    bk_y.value = change["new"]
def change_ip_z(change):
    bk_z.value = change["new"]
ip_x.observe(change_ip_x, names="value")
ip_y.observe(change_ip_y, names="value")
ip_z.observe(change_ip_z, names="value")

bk_hbox = row([bk_vbox, bk_plot])
ip_hbox = ipw.HBox([ip_vbox, ipv_plot])

wrapper = IPyWidget(widget=ip_hbox, width=800, height=300)
layout = column([bk_hbox, wrapper])

doc = curdoc()
doc.add_root(layout)

In [38]:
import datetime
from os.path import dirname, join

import pandas as pd
from scipy.signal import savgol_filter

from bokeh.io import curdoc
from bokeh.layouts import column, row
from bokeh.models import ColumnDataSource, DataRange1d, Select
from bokeh.palettes import Blues4
from bokeh.plotting import figure

STATISTICS = ['record_min_temp', 'actual_min_temp', 'average_min_temp', 'average_max_temp', 'actual_max_temp', 'record_max_temp']

def get_dataset(src, name, distribution):
    df = src[src.airport == name].copy()
    del df['airport']
    df['date'] = pd.to_datetime(df.date)
    # timedelta here instead of pd.DateOffset to avoid pandas bug < 0.18 (Pandas issue #11925)
    df['left'] = df.date - datetime.timedelta(days=0.5)
    df['right'] = df.date + datetime.timedelta(days=0.5)
    df = df.set_index(['date'])
    df.sort_index(inplace=True)
    if distribution == 'Smoothed':
        window, order = 51, 3
        for key in STATISTICS:
            df[key] = savgol_filter(df[key], window, order)

    return ColumnDataSource(data=df)

def make_plot(source, title):
    plot = figure(x_axis_type="datetime", width=800, tools="", toolbar_location=None)
    plot.title.text = title

    plot.quad(top='record_max_temp', bottom='record_min_temp', left='left', right='right',
              color=Blues4[2], source=source, legend_label="Record")
    plot.quad(top='average_max_temp', bottom='average_min_temp', left='left', right='right',
              color=Blues4[1], source=source, legend_label="Average")
    plot.quad(top='actual_max_temp', bottom='actual_min_temp', left='left', right='right',
              color=Blues4[0], alpha=0.5, line_color="black", source=source, legend_label="Actual")

    # fixed attributes
    plot.xaxis.axis_label = None
    plot.yaxis.axis_label = "Temperature (F)"
    plot.axis.axis_label_text_font_style = "bold"
    plot.x_range = DataRange1d(range_padding=0.0)
    plot.grid.grid_line_alpha = 0.3

    return plot

def update_plot(attrname, old, new):
    city = city_select.value
    plot.title.text = "Weather data for " + cities[city]['title']

    src = get_dataset(df, cities[city]['airport'], distribution_select.value)
    source.data.update(src.data)

city = 'Austin'
distribution = 'Discrete'

cities = {
    'Austin': {
        'airport': 'AUS',
        'title': 'Austin, TX',
    },
    'Boston': {
        'airport': 'BOS',
        'title': 'Boston, MA',
    },
    'Seattle': {
        'airport': 'SEA',
        'title': 'Seattle, WA',
    }
}

city_select = Select(value=city, title='City', options=sorted(cities.keys()))
distribution_select = Select(value=distribution, title='Distribution', options=['Discrete', 'Smoothed'])

df = pd.read_csv('data/001_sample.csv')
source = get_dataset(df, cities[city]['airport'], distribution)
plot = make_plot(source, "Weather data for " + cities[city]['title'])

city_select.on_change('value', update_plot)
distribution_select.on_change('value', update_plot)

controls = column(city_select, distribution_select)

curdoc().add_root(row(plot, controls))
curdoc().title = "Weather"

RuntimeError: _pending_writes should be non-None when we have a document lock, and we should have the lock when the document changes